In [ ]:
# Handwritten Digit Recognition (CNN)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split

# Load and preprocess data

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[:12500]
y_train = y_train[:12500]
x_test = x_test[:2500]
y_test = y_test[:2500]

# Normalize
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

print("Training samples:", x_train.shape[0])
print("Testing samples:", x_test.shape[0])


# Data augmentation

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

x_train = x_train.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)
datagen.fit(x_train)

# CNN model

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# Split manually
x_train_part, x_val, y_train_part, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

history = model.fit(
    datagen.flow(x_train_part, y_train_part, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=15,
    verbose=1
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend(); plt.title('Loss')
plt.show()

# Google colab upload

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    img_path = fn
    img = load_img(img_path, color_mode='grayscale', target_size=(28,28))
    img = img_to_array(img)
    img = 255 - img  # invert if white background
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # add batch dim
    prediction = np.argmax(model.predict(img))

    plt.imshow(img.reshape(28,28), cmap='gray')
    plt.title(f"Predicted Digit: {prediction}")
    plt.axis('off')
    plt.show()

model.save("handwritten_digit_ml.keras")
print(" Model saved as handwritten_digit_ml.keras")


RUNNING ONLY


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab import files
import matplotlib.pyplot as plt

model = load_model("handwritten_digit_ml.keras")

input_size = model.input_shape[1:3]

while True:
    choice = input("\nUpload an image? (y/n): ")
    if choice.lower() != 'y':
        print("Exiting testing loop.")
        break

    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"\nProcessing file: {filename}")

        img = cv2.imread(filename)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if np.mean(gray) > 127:
            gray = 255 - gray
        _, bw = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
        bw = cv2.resize(bw, input_size)

        bw = bw.astype('float32') / 255.0
        bw = bw.reshape(1, input_size[0], input_size[1], 1)

        pred = np.argmax(model.predict(bw), axis=-1)[0]
        print(f"✅ Predicted Digit: {pred}")

        plt.imshow(bw.reshape(input_size[0], input_size[1]), cmap='gray')
        plt.title(f"Predicted Digit: {pred}")
        plt.axis('off')
        plt.show()
